# Clustering and annotation of combined disease network propagation subnetworks (z>2)

In [1]:
# import matplotlib
# matplotlib.use('TkAgg')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pandas as pd
import random

import community

from scipy.stats import mannwhitneyu

import mygene
mg = mygene.MyGeneInfo()

# latex rendering of text in graphs
import matplotlib as mpl
mpl.rc('text', usetex = False)
mpl.rc('font', family = 'serif')

sns.set_style('white')

import sys

sys.path.append('../../code/')
import load_interactomes


import visJS2jupyter.visJS_module
import visJS2jupyter.visualizations



% matplotlib inline

# Load the ASD-CHD interactome

In [2]:
G_ASD_CHD = nx.read_gpickle('combined_disease_networks/G_ASD_CHD_z2_before5000_repsGIANT_p2.gpickle')
print(len(G_ASD_CHD.nodes()))
print(len(G_ASD_CHD.edges()))

1512
40828


In [27]:
len(np.unique(G_ASD_CHD.nodes()))

1512

In [28]:
Gc = max(nx.connected_component_subgraphs(G_ASD_CHD), key=len)
len(Gc.nodes())

1503

In [3]:
# compute the louvain clusters
partition = community.best_partition(G_ASD_CHD)

In [4]:
partition=pd.Series(partition)
partition.value_counts()

1     658
0     315
3     263
2     247
5       9
6       3
8       3
12      3
16      2
10      1
9       1
7       1
11      1
4       1
13      1
14      1
15      1
17      1
dtype: int64

In [7]:
nx.set_node_attributes(G_ASD_CHD,'partition',dict(partition))


In [10]:
# write the nodelist and edgelists
[e1,e2] = zip(*G_ASD_CHD.edges())
w = pd.Series(nx.get_edge_attributes(G_ASD_CHD,'weight')).loc[G_ASD_CHD.edges()].tolist()
edge_info = pd.DataFrame({'edge1':e1,'edge2':e2,'weight':w})
len(edge_info)
edge_info.to_csv('G_ASD_CHD_180925.tsv',sep='\t') # save the edges




In [17]:
node_table = pd.DataFrame(np.zeros((len(G_ASD_CHD.nodes()),6)),index=G_ASD_CHD.nodes())
node_table.columns = ['ASD_HC','CHD_HC','partition','z_ASD','z_ASD_CHD','z_CHD']

for node in G_ASD_CHD.nodes():
    node_table.loc[node] = pd.Series(G_ASD_CHD.node[node])
    
node_table.to_csv('G_ASD_CHD_180925_nodes.tsv',sep='\t') # save the nodes
node_table.head()

,ASD_HC,CHD_HC,partition,z_ASD,z_ASD_CHD,z_CHD
PGM2L1,0.0,0.0,0.0,2.105800,3.023162,2.644882
NIPA1,0.0,0.0,0.0,2.137929,2.233875,0.874721
LIFR,0.0,0.0,1.0,2.167702,2.907379,2.019243
PIK3CA,0.0,0.0,2.0,2.650072,2.363422,0.943191
ZNF701,0.0,0.0,0.0,1.984758,2.626359,1.563654


In [26]:
node_table=node_table.sort_values('z_ASD_CHD',ascending=False)
node_table.head(177).tail()

,ASD_HC,CHD_HC,partition,z_ASD,z_ASD_CHD,z_CHD
TMTC3,0.0,0.0,0.0,4.318288,3.787568,0.004409
KLHL2,0.0,0.0,2.0,3.366697,3.781594,2.385985
KANSL1L,0.0,0.0,2.0,3.206343,3.774944,2.351072
ZC3H7A,0.0,0.0,2.0,6.682980,3.766292,0.335452
ATP8B1,0.0,0.0,1.0,3.116581,3.761816,2.114954


In [25]:
(node_table[['ASD_HC','CHD_HC']].sum(axis=1)>0).sum()

77

In [6]:
# annotate each cluster with gprofiler
from gprofiler import GProfiler
gp = GProfiler("MyToolName/0.1")

import os

savedir = 'gprofile_clusters_ASD_CHD'
if not os.path.isdir(savedir):
    os.mkdir(savedir)


for focal_cluster in partition.value_counts().index.tolist():
    print(focal_cluster)
    focal_genes = partition[partition==focal_cluster].index.tolist()

    if len(focal_genes)>10: # only save results if there are enough genes
        gp_results = pd.DataFrame(gp.gprofile(focal_genes,custom_bg = partition.index.tolist(),correction_method=gp.THR_FDR))

        gp_results.columns = ["query.number", "significant", "p.value", "term.size",
                              "query.size", "overlap.size", "recall", "precision",
                              "term.id", "domain", "subgraph.number", "term.name",
                              "relative.depth", "intersection"]
        print(gp_results[['p.value','term.id','term.name']].head())

        writer = pd.ExcelWriter(savedir+'/cluster_'+str(focal_cluster)+'.xlsx')
        gp_results.to_excel(writer)
        writer.save()

#gp_results = pd.DataFrame(gp.gprofile(focal_genes,correction_method=gp.THR_FDR))

1
        p.value     term.id                               term.name
0  1.520000e-21  GO:0071944                          cell periphery
1  2.070000e-21  GO:0005886                         plasma membrane
2  1.060000e-16  GO:0044459                    plasma membrane part
3  1.190000e-12  GO:0044425                           membrane part
4  1.440000e-11  GO:0031226  intrinsic component of plasma membrane
0
    p.value            term.id                        term.name
0  0.000491   REAC:R-HSA-74160  Gene expression (Transcription)
1  0.003350         GO:0016740             transferase activity
2  0.003430         GO:0003676             nucleic acid binding
3  0.005810         GO:0031981                    nuclear lumen
4  0.010700  REAC:R-HSA-212436    Generic Transcription Pathway
3
    p.value             term.id                                      term.name
0  0.000094  REAC:R-HSA-4839726                         Chromatin organization
1  0.000094  REAC:R-HSA-3247509             

In [29]:
for g in partition[partition==2].index.tolist():
    print(g)

644504
ABHD3
ACAP2
ACER3
ACOT9
ACVR1
ADNP
AP1G1
APPBP2
ARAP2
ATP11B
ATP2B1
AVL9
BAGE2
BAZ2B
BPGM
BTAF1
C2CD5
CAMSAP2
CAPZA1
CASC15
CAST
CCDC117
CCDC126
CCP110
CCSER2
CCT8
CELF2
CEP170
CEP192
CHD1
CHD9
CHIC2
CIR1
CITED2
CLASP2
CLIP1
CLK1
CNOT6
COG6
COP1
CPEB3
CRBN
CSGALNACT2
CTTNBP2
CUL3
DAAM1
DCAF17
DCUN1D1
DMXL2
DNAJC13
DOPEY1
DSE
E2F3
EED
EFR3A
EID3
ELF2
ELMOD1
ERBIN
FAM172A
FAM217B
FAM8A1
FBXO11
FBXO33
FBXO34
FBXW11
FCHSD2
FMR1
FZD3
GGNBP2
GOLPH3L
GPCPD1
HECA
HIVEP2
HMGXB4
HNRNPH3
HOMER1
IBTK
IL17RD
IL1RAP
ILF2
IQGAP1
IRF2
IVNS1ABP
JAK1
JAK2
JMJD1C
JMY
KANSL1L
KAT2B
KBTBD2
KDM6A
KIF6
KLHL2
KRAS
LEMD3
LNPK
LNX2
LRP12
LRRFIP2
LSM14A
MAP3K5
MAP4K5
MARCH5
MARCH7
MARCH8
MBNL1
MDFIC
MED13
MEF2A
MEX3C
MFAP3
MIR936
MORC3
NAB1
NBN
NCKAP1
NDEL1
NEK7
NFE2L2
NGLY1
NIPBL
NR3C1
NRIP1
NUS1P2
OSBPL8
OXR1
PDCD4
PDS5B
PELI1
PEX3
PHIP
PI4KAP2
PIK3CA
PIKFYVE
PLAGL1
PPM1A
PPM1B
PPM1D
PPP1R12A
PPP2CB
PRKD3
PSME4
PSPC1
PTBP2
PTEN
PTENP1
PTPN11
QKI
RAB11FIP2
RAB33B
RABGAP1L
RALBP1
RALGAPB
RAP1A
RAPGEF2
RB1

# Repeat the process for ASD-EPI

In [30]:
G_ASD_EPI = nx.read_gpickle('combined_disease_networks/G_ASD_EPI_z2_before5000_repsGIANT_p2.gpickle')
print(len(G_ASD_EPI.nodes()))
print(len(G_ASD_EPI.edges()))

1320
14993


In [31]:
# compute the louvain clusters
partition = community.best_partition(G_ASD_EPI)

In [32]:
partition = pd.Series(partition)
partition.value_counts()

0     459
1     262
4     245
2     231
3     108
5       3
13      2
12      2
9       2
6       2
11      1
10      1
8       1
7       1
dtype: int64

In [33]:
nx.set_node_attributes(G_ASD_EPI,'partition',dict(partition))

In [35]:
# write the nodelist and edgelists
[e1,e2] = zip(*G_ASD_EPI.edges())
w = pd.Series(nx.get_edge_attributes(G_ASD_EPI,'weight')).loc[G_ASD_EPI.edges()].tolist()
edge_info = pd.DataFrame({'edge1':e1,'edge2':e2,'weight':w})
len(edge_info)
edge_info.to_csv('combined_disease_networks/G_ASD_EPI_180925.tsv',sep='\t') # save the edges




In [37]:
G_ASD_EPI.nodes()[0]

'HIF3A'

In [39]:
pd.Series(G_ASD_EPI.node['HIF3A']).loc[['ASD_HC','EPI_HC','partition','z_ASD','z_ASD_EPI','z_EPI']]

ASD_HC       0.000000
EPI_HC       0.000000
partition    0.000000
z_ASD        2.235018
z_ASD_EPI    2.187273
z_EPI        1.110274
dtype: float64

In [41]:
node_table = pd.DataFrame(np.zeros((len(G_ASD_EPI.nodes()),6)),index=G_ASD_EPI.nodes())
node_table.columns = ['ASD_HC','EPI_HC','partition','z_ASD','z_ASD_EPI','z_EPI']

for node in G_ASD_EPI.nodes():
    node_table.loc[node] = pd.Series(G_ASD_EPI.node[node]).loc[['ASD_HC','EPI_HC','partition','z_ASD','z_ASD_EPI','z_EPI']]
    
node_table.to_csv('combined_disease_networks/G_ASD_EPI_180925_nodes.tsv',sep='\t') # save the nodes
node_table.head()

,ASD_HC,EPI_HC,partition,z_ASD,z_ASD_EPI,z_EPI
HIF3A,0.0,0.0,0.0,2.235018,2.187273,1.110274
ERRFI1,0.0,0.0,1.0,2.870195,2.806272,0.416803
PHYHIPL,0.0,0.0,0.0,2.875986,3.809215,2.628966
ADAM23,0.0,0.0,0.0,1.343268,2.659139,2.284904
PGM2L1,0.0,0.0,1.0,2.105800,2.212774,0.692421


In [42]:
savedir = 'gprofile_clusters_ASD_EPI'
if not os.path.isdir(savedir):
    os.mkdir(savedir)


for focal_cluster in partition.value_counts().index.tolist():
    print(focal_cluster)
    focal_genes = partition[partition==focal_cluster].index.tolist()

    if len(focal_genes)>10: # only save results if there are enough genes
        gp_results = pd.DataFrame(gp.gprofile(focal_genes,custom_bg = partition.index.tolist(),correction_method=gp.THR_FDR))

        gp_results.columns = ["query.number", "significant", "p.value", "term.size",
                              "query.size", "overlap.size", "recall", "precision",
                              "term.id", "domain", "subgraph.number", "term.name",
                              "relative.depth", "intersection"]
        print(gp_results[['p.value','term.id','term.name']].head())

        writer = pd.ExcelWriter(savedir+'/cluster_'+str(focal_cluster)+'.xlsx')
        gp_results.to_excel(writer)
        writer.save()

#gp_results = pd.DataFrame(gp.gprofile(focal_genes,correction_method=gp.THR_FDR))

0
        p.value     term.id                            term.name
0  5.910000e-13  GO:0022839           ion gated channel activity
1  1.040000e-12  GO:0022836               gated channel activity
2  1.820000e-11  GO:0005216                 ion channel activity
3  2.410000e-11  GO:0022838  substrate-specific channel activity
4  2.410000e-11  GO:0015267                     channel activity
1
        p.value     term.id                term.name
0  3.980000e-12  GO:0005654              nucleoplasm
1  2.130000e-11  GO:0044428             nuclear part
2  2.520000e-11  GO:0031981            nuclear lumen
3  6.430000e-10  GO:0051276  chromosome organization
4  2.870000e-09  GO:0031974  membrane-enclosed lumen
4
    p.value      term.id                                          term.name
0  0.000668  TF:M01388_1  Factor: Dlx-5; motif: NNRGYAATTRNYKNNN; match ...
1  0.000668    TF:M01388             Factor: Dlx-5; motif: NNRGYAATTRNYKNNN
2  0.000668  TF:M01388_0  Factor: Dlx-5; motif: NNRGYAATTR

# Repeat for EPI-CHD

In [43]:
G_EPI_CHD = nx.read_gpickle('combined_disease_networks/G_EPI_CHD_z2_before5000_repsGIANT_p2.gpickle')
print(len(G_EPI_CHD.nodes()))
print(len(G_EPI_CHD.edges()))

852
4903


In [47]:
# compute the louvain clusters
partition = community.best_partition(G_EPI_CHD)

In [48]:
partition = pd.Series(partition)
partition.value_counts()

3     266
1     197
2     154
0     105
7      70
6      30
5      20
9       3
14      1
13      1
12      1
11      1
10      1
8       1
4       1
dtype: int64

In [49]:
nx.set_node_attributes(G_EPI_CHD,'partition',dict(partition))

In [50]:
# write the nodelist and edgelists
[e1,e2] = zip(*G_EPI_CHD.edges())
w = pd.Series(nx.get_edge_attributes(G_EPI_CHD,'weight')).loc[G_EPI_CHD.edges()].tolist()
edge_info = pd.DataFrame({'edge1':e1,'edge2':e2,'weight':w})
len(edge_info)
edge_info.to_csv('combined_disease_networks/G_G_EPI_CHD_180925.tsv',sep='\t') # save the edges




In [51]:
node_table = pd.DataFrame(np.zeros((len(G_EPI_CHD.nodes()),6)),index=G_EPI_CHD.nodes())
node_table.columns = ['CHD_HC','EPI_HC','partition','z_CHD','z_EPI_CHD','z_EPI']

for node in G_EPI_CHD.nodes():
    node_table.loc[node] = pd.Series(G_EPI_CHD.node[node]).loc[['CHD_HC','EPI_HC','partition','z_CHD','z_EPI_CHD','z_EPI']]
    
node_table.to_csv('combined_disease_networks/G_EPI_CHD_180925_nodes.tsv',sep='\t') # save the nodes
node_table.head()

,CHD_HC,EPI_HC,partition,z_CHD,z_EPI_CHD,z_EPI
HSPA2,0.0,0.0,0.0,2.445433,2.288136,0.963411
TLCD1,0.0,0.0,1.0,-0.018130,2.390685,3.920589
ARMC9,0.0,0.0,1.0,1.087595,2.385754,2.243174
PGM2L1,0.0,0.0,2.0,2.644882,2.207897,0.692421
TPD52L1,0.0,0.0,1.0,4.032180,2.798880,0.204820


In [53]:
savedir = 'gprofile_clusters_EPI_CHD'
if not os.path.isdir(savedir):
    os.mkdir(savedir)


for focal_cluster in partition.value_counts().index.tolist():
    print(focal_cluster)
    focal_genes = partition[partition==focal_cluster].index.tolist()

    if len(focal_genes)>10: # only save results if there are enough genes
        gp_results = pd.DataFrame(gp.gprofile(focal_genes,custom_bg = partition.index.tolist(),correction_method=gp.THR_FDR))

        gp_results.columns = ["query.number", "significant", "p.value", "term.size",
                              "query.size", "overlap.size", "recall", "precision",
                              "term.id", "domain", "subgraph.number", "term.name",
                              "relative.depth", "intersection"]
        print(gp_results[['p.value','term.id','term.name']].head())

        writer = pd.ExcelWriter(savedir+'/cluster_'+str(focal_cluster)+'.xlsx')
        gp_results.to_excel(writer)
        writer.save()

#gp_results = pd.DataFrame(gp.gprofile(focal_genes,correction_method=gp.THR_FDR))

3
    p.value     term.id                             term.name
0  0.000041  GO:0098916  anterograde trans-synaptic signaling
1  0.000041  GO:0097458                           neuron part
2  0.000041  GO:0007268        chemical synaptic transmission
3  0.000041  GO:0022836                gated channel activity
4  0.000041  GO:0099537              trans-synaptic signaling
1
        p.value             term.id                       term.name
0  2.120000e-08          GO:0043588                skin development
1  2.120000e-08          GO:0008544           epidermis development
2  4.440000e-07          GO:0030216    keratinocyte differentiation
3  5.530000e-07          GO:0009913  epidermal cell differentiation
4  2.770000e-05  REAC:R-HSA-6805567                  Keratinization
2
   p.value      term.id                                          term.name
0  0.00181   GO:0070647  protein modification by small protein conjugat...
1  0.00298   GO:0051276                            chromosome or